In [ ]:
import os
import sys
import h5py
import glob
import json
import math
import time
import timeit
import hdbscan
import itertools
import importlib
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from scipy import stats
import keras.backend as K
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import DBSCAN, KMeans
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, spearmanr
from sklearn.preprocessing import normalize, scale, MinMaxScaler, StandardScaler
from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram, linkage, fcluster

from tcn import TCN
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import svm, datasets
from IPython.display import Image
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.metrics import Precision, Recall
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics.cluster import normalized_mutual_info_score
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.model_selection import train_test_split, TimeSeriesSplit, KFold
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, BatchNormalization, Activation, GlobalAveragePooling1D, Dense, add, Dropout, concatenate, LSTM
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, log_loss, classification_report, silhouette_score

In [ ]:
# calculate statistics

import os
import pandas as pd
import numpy as np

# Define the file structure and model names
base_path = ''
metrics = {
    'f1': 'f1_score',
    'roc': 'roc_auc_score',
    'mcc': 'mcc'
}
methods = ['wf', 'bw', 'sw']
models = ['XGBoost', 'RandomForest', 'SupportVectorMachine', 'ROCKET', 'InceptionTime', 'LSTM_FCN', 'ResNet', 'TCN']
k_folds = [3, 4, 5, 6, 7, 8, 9]

# Initialize an empty list to store the results
results_list = []

# Loop through each metric, method, model, and k-fold to read CSV files and calculate statistics
for metric, column_name in metrics.items():
    for method in methods:
        for model in models:
            for k in k_folds:
                file_name = f'{model}_splits_{k}.csv'
                file_path = os.path.join(base_path, metric, method, file_name)

                if os.path.exists(file_path):
                    try:
                        # Read the CSV file
                        data = pd.read_csv(file_path)
                        
                        # Check if the data is not empty
                        if not data.empty:
                            # Calculate the statistics for the specified column
                            ave = data[column_name].mean()
                            std = data[column_name].std(ddof=0)
                            med = data[column_name].median()

                            # Append the results to the list
                            results_list.append({
                                'Model': model,
                                'K-fold': k,
                                'TSCV Method': method,
                                'Metric': metric.upper(),
                                'AVE': ave,
                                'STD': std,
                                'MED': med
                            })
                    except pd.errors.EmptyDataError:
                        print(f"File {file_path} is empty and will be skipped.")

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results_list, columns=['Model', 'K-fold', 'TSCV Method', 'Metric', 'AVE', 'STD', 'MED'])

# Save the results to a new CSV file
results_df.to_csv()
print("Results have been saved to 'benchmark_results.csv'")

# Rankings

## Classifier ranks

In [ ]:
def rank_models_mcc(df, k_folds):
    results = []
    
    for k in k_folds:
        k_df = df[df['K-fold'] == k]
        
        for split in k_df['split'].unique():
            split_df = k_df[k_df['split'] == split].sort_values(by='mcc', ascending=False).reset_index(drop=True)
            split_df['rank'] = split_df.index + 1
            split_df['points'] = split_df['rank']  # Best model gets the lowest points
            results.append(split_df)
    
    result_df = pd.concat(results)
    
    # Calculate the average points for each model across all splits for each k-fold
    avg_points_df = result_df.groupby(['Model', 'K-fold'])['points'].mean().reset_index()
    avg_points_df = avg_points_df.sort_values(by='points', ascending=True)
    
    return avg_points_df

def rank_models_roc(df, k_folds):
    results = []
    
    for k in k_folds:
        k_df = df[df['K-fold'] == k]
        
        for split in k_df['split'].unique():
            split_df = k_df[k_df['split'] == split].sort_values(by='roc_auc_score', ascending=False).reset_index(drop=True)
            split_df['rank'] = split_df.index + 1
            split_df['points'] = split_df['rank']  # Best model gets the lowest points
            results.append(split_df)
    
    result_df = pd.concat(results)
    
    # Calculate the average points for each model across all splits for each k-fold
    avg_points_df = result_df.groupby(['Model', 'K-fold'])['points'].mean().reset_index()
    avg_points_df = avg_points_df.sort_values(by='points', ascending=True)
    
    return avg_points_df

def rank_models_f1(df, k_folds):
    results = []
    
    for k in k_folds:
        k_df = df[df['K-fold'] == k]
        
        for split in k_df['split'].unique():
            split_df = k_df[k_df['split'] == split].sort_values(by='f1_score', ascending=False).reset_index(drop=True)
            split_df['rank'] = split_df.index + 1
            split_df['points'] = split_df['rank']  # Best model gets the lowest points
            results.append(split_df)
    
    result_df = pd.concat(results)
    
    # Calculate the average points for each model across all splits for each k-fold
    avg_points_df = result_df.groupby(['Model', 'K-fold'])['points'].mean().reset_index()
    avg_points_df = avg_points_df.sort_values(by='points', ascending=True)
    
    return avg_points_df

k_folds = [3, 4, 5, 6, 7, 8, 9]

path_mcc = r'/mcc/'
wf_mcc_df = pd.read_csv(path_mcc + 'SortedMCC_tscv_wf.csv')
bw_mcc_df = pd.read_csv(path_mcc + 'SortedMCC_tscv_bw.csv')
sw_mcc_df = pd.read_csv(path_mcc + 'SortedMCC_tscv_sw.csv')

wf_mcc_avg_points = rank_models_mcc(wf_mcc_df, k_folds)
bw_mcc_avg_points = rank_models_mcc(bw_mcc_df, k_folds)
sw_mcc_avg_points = rank_models_mcc(sw_mcc_df, k_folds)

# Save the results to CSV files
wf_mcc_avg_points.to_csv(path_mcc + 'wf_avg_points.csv', index=False)
bw_mcc_avg_points.to_csv(path_mcc + 'bw_avg_points.csv', index=False)
sw_mcc_avg_points.to_csv(path_mcc + 'sw_avg_points.csv', index=False)

path_roc = r'/roc/'
wf_roc_df = pd.read_csv(path_roc + 'SortedROC_tscv_wf.csv')
bw_roc_df = pd.read_csv(path_roc + 'SortedROC_tscv_bw.csv')
sw_roc_df = pd.read_csv(path_roc + 'SortedROC_tscv_sw.csv')

wf_roc_avg_points = rank_models_roc(wf_roc_df, k_folds)
bw_roc_avg_points = rank_models_roc(bw_roc_df, k_folds)
sw_roc_avg_points = rank_models_roc(sw_roc_df, k_folds)

# Save the results to CSV files
wf_roc_avg_points.to_csv(path_roc + 'wf_avg_points.csv', index=False)
bw_roc_avg_points.to_csv(path_roc + 'bw_avg_points.csv', index=False)
sw_roc_avg_points.to_csv(path_roc + 'sw_avg_points.csv', index=False)

path_f1 = r'/f1/'
wf_f1_df = pd.read_csv(path_f1 + 'SortedF1_tscv_wf.csv')
bw_f1_df = pd.read_csv(path_f1 + 'SortedF1_tscv_bw.csv')
sw_f1_df = pd.read_csv(path_f1 + 'SortedF1_tscv_sw.csv')

wf_f1_avg_points = rank_models_f1(wf_f1_df, k_folds)
bw_f1_avg_points = rank_models_f1(bw_f1_df, k_folds)
sw_f1_avg_points = rank_models_f1(sw_f1_df, k_folds)

# Display the results
print("Walk-Forward (WF) Average Points")
print(wf_f1_avg_points)
print("\nBlocking Window (BW) Average Points")
print(bw_f1_avg_points)
print("\nSliding Window (SW) Average Points")
print(sw_f1_avg_points)

# Save the results to CSV files
wf_f1_avg_points.to_csv(path_f1 + 'wf_avg_points.csv', index=False)
bw_f1_avg_points.to_csv(path_f1 + 'bw_avg_points.csv', index=False)
sw_f1_avg_points.to_csv(path_f1 + 'sw_avg_points.csv', index=False)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

path1 = r'/f1/'
path2 = r'/roc/'
path3 = r'/mcc/'

# Load the ranking CSV files
wf_f1_df = pd.read_csv(path1+'wf_avg_points.csv')
wf_roc_df = pd.read_csv(path2+'wf_avg_points.csv')
wf_mcc_df = pd.read_csv(path3+'wf_avg_points.csv')

bw_f1_df = pd.read_csv(path1+'bw_avg_points.csv')
bw_roc_df = pd.read_csv(path2+'bw_avg_points.csv')
bw_mcc_df = pd.read_csv(path3+'bw_avg_points.csv')

sw_f1_df = pd.read_csv(path1+'sw_avg_points.csv')
sw_roc_df = pd.read_csv(path2+'sw_avg_points.csv')
sw_mcc_df = pd.read_csv(path3+'sw_avg_points.csv')

# Combine DataFrames for each TSCV method
wf_df = pd.concat([wf_f1_df.assign(Metric='F1'), wf_roc_df.assign(Metric='ROC'), wf_mcc_df.assign(Metric='MCC')])
bw_df = pd.concat([bw_f1_df.assign(Metric='F1'), bw_roc_df.assign(Metric='ROC'), bw_mcc_df.assign(Metric='MCC')])
sw_df = pd.concat([sw_f1_df.assign(Metric='F1'), sw_roc_df.assign(Metric='ROC'), sw_mcc_df.assign(Metric='MCC')])

# Plotting Function
def plot_ranking(df, method):
    plt.figure(figsize=(14, 8))
    
    # Sort data by points in descending order within each k-fold
    df = df.sort_values(by=['K-fold', 'points'], ascending=[True, False])
    
    sns.barplot(x='K-fold', y='points', ci=None, hue='Model', data=df, dodge=True)
    plt.title(f'{method} Method - Model Ranking by Average Points')
    plt.xlabel('K-fold')
    plt.ylabel('Average Points')
    plt.legend(loc='upper right')
    plt.show()

# Plot rankings for each TSCV method
plot_ranking(wf_df, 'Walk-Forward (WF)')
plot_ranking(bw_df, 'Blocking Window (BW)')
plot_ranking(sw_df, 'Sliding Window (SW)')


In [ ]:
from matplotlib.lines import Line2D
# Combine DataFrames for each TSCV method
wf_df = pd.concat([wf_f1_avg_points.assign(Metric='F1'), wf_roc_avg_points.assign(Metric='ROC'), wf_mcc_avg_points.assign(Metric='MCC')])
bw_df = pd.concat([bw_f1_avg_points.assign(Metric='F1'), bw_roc_avg_points.assign(Metric='ROC'), bw_mcc_avg_points.assign(Metric='MCC')])
sw_df = pd.concat([sw_f1_avg_points.assign(Metric='F1'), sw_roc_avg_points.assign(Metric='ROC'), sw_mcc_avg_points.assign(Metric='MCC')])

# Combine all DataFrames for easier plotting
combined_df = pd.concat([wf_df.assign(Method='WF'), bw_df.assign(Method='BW'), sw_df.assign(Method='SW')])

# Define models and their respective markers
models = combined_df['Model'].unique()
colors = ['#d7191c', '#fdae61', '#a6d96a', '#1a9641', '#2c7bb6', '#abd9e9', '#fdae61', '#f46d43']
markers = ['o', 's', 'd', '^', 'v', '<', '>', 'p']

# Define k-folds
k_folds = [3, 4, 5, 6, 7, 8, 9]

# xytext offsets for annotations
xytext_offsets = {
    (0, 2, 4): (-10, -45),
}

# Function to get xytext offset for each annotation
def get_xytext_offset(key):
    return xytext_offsets.get(key, (0, -45))

def plot_ranking(df, k_folds, models, metrics, colors, markers):
    fig, axs = plt.subplots(1, len(k_folds), figsize=(25, 10), sharey=True)
    
    for idx, k in enumerate(k_folds):
        k_df = df[df['K-fold'] == k]
        ranking_df = []
        
        for method in ['WF', 'BW', 'SW']:
            for metric in metrics:
                metric_df = k_df[(k_df['Method'] == method) & (k_df['Metric'] == metric)].sort_values(by='points', ascending=True)
                metric_df['Rank'] = range(1, len(metric_df) + 1)
                metric_df['Method_Metric'] = f'{method}-{metric}'
                ranking_df.append(metric_df)
        
        ranking_df = pd.concat(ranking_df)
        ranking_df['Method_Metric'] = pd.Categorical(ranking_df['Method_Metric'], categories=[
            f'WF-{metric}' for metric in metrics] + [f'BW-{metric}' for metric in metrics] + [f'SW-{metric}' for metric in metrics], ordered=True)
        
        for model in models:
            model_df = ranking_df[ranking_df['Model'] == model]
            axs[idx].semilogx(model_df['points'], model_df['Method_Metric'], label=model, marker=markers[np.where(models == model)[0][0]], color=colors[np.where(models == model)[0][0]], linestyle='', markersize=10)
            # Add annotations with actual values
            for j, row in model_df.iterrows():
                xytext = get_xytext_offset((idx, np.where(models == model)[0][0], j))
                axs[idx].annotate(f'{row["points"]:.2f}', (row['points'], row['Method_Metric']), textcoords="offset points", xytext=xytext, ha='center', fontsize=9, color=colors[np.where(models == model)[0][0]])
                # Temp labels for identification, spaced out vertically
                #axs[i].annotate(f'{i}-{j}-{k}', (data[j, k], k), rotation=0, textcoords="offset points", xytext=(0, 25 + j * 15), ha='center', fontsize=12, color='blue', bbox=dict(facecolor='yellow', alpha=0.7))
        
        axs[idx].set_xscale('log')
        axs[idx].set_xlim(2.2, 7)  # Adjusted to fit the data better
        axs[idx].set_xticks([3, 3.5, 4, 4.5, 5, 5.5, 6])
        axs[idx].set_xticklabels(['3', '3.5', '4', '4.5', '5', '5.5', '6'], fontsize=12)
        axs[idx].set_title(f'K-fold = {k}', fontsize=20)
        axs[idx].set_xlabel('Average Points (Log Scale)', fontsize=15)
        axs[idx].invert_yaxis()  # Highest rank (1) at the top
        
    axs[0].set_ylabel('Method-Metric', fontsize=15)
    handles = [Line2D([0], [0], color=color, marker=marker, linestyle='', markersize=14) for color, marker in zip(colors, markers)]
    fig.legend(handles, models, fontsize=17, loc='upper center', ncol=8, bbox_to_anchor=(0.5, 1.05))

    plt.tight_layout()
    plt.show()

# Define metrics
metrics = ['F1', 'ROC', 'MCC']

# Plot rankings for each k-fold
plot_ranking(combined_df, k_folds, models, metrics, colors, markers)

In [ ]:
# Changing the plot

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D

# Combine DataFrames for each TSCV method
wf_df = pd.concat([wf_f1_avg_points.assign(Metric='F1'), wf_roc_avg_points.assign(Metric='ROC'), wf_mcc_avg_points.assign(Metric='MCC')])
bw_df = pd.concat([bw_f1_avg_points.assign(Metric='F1'), bw_roc_avg_points.assign(Metric='ROC'), bw_mcc_avg_points.assign(Metric='MCC')])
sw_df = pd.concat([sw_f1_avg_points.assign(Metric='F1'), sw_roc_avg_points.assign(Metric='ROC'), sw_mcc_avg_points.assign(Metric='MCC')])

# Combine all DataFrames for easier plotting
combined_df = pd.concat([wf_df.assign(Method='WF'), bw_df.assign(Method='BW'), sw_df.assign(Method='SW')])

# Define models and their respective markers
models = combined_df['Model'].unique()
colors = ['#d7191c', '#fdae61', '#a6d96a', '#1a9641', '#2c7bb6', '#abd9e9', '#fdae61', '#f46d43']
markers = ['o', 's', 'd', '^', 'x', '*', '_', '+']

# Define k-folds
k_folds = [3, 4, 5, 6, 7, 8, 9]

def plot_ranking(df, method, models, metrics, colors, markers):
    fig, axs = plt.subplots(3, 1, figsize=(20, 30), sharex=True)
    
    for metric_idx, metric in enumerate(metrics):
        metric_df = df[(df['Method'] == method) & (df['Metric'] == metric)]
        for model in models:
            model_df = metric_df[metric_df['Model'] == model]
            axs[metric_idx].semilogx(model_df['points'], model_df['K-fold'], label=model, marker=markers[np.where(models == model)[0][0]], color=colors[np.where(models == model)[0][0]], linestyle='', markersize=12, alpha=0.7)
            # Add annotations with actual values
            for j, row in model_df.iterrows():
                axs[metric_idx].annotate(f'{row["points"]:.1f}', (row['points'], row['K-fold']), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=12, color=colors[np.where(models == model)[0][0]])
        
        axs[metric_idx].set_xscale('log')
        axs[metric_idx].set_xlim(1.1, 7)  
        axs[metric_idx].set_xticks([3, 3.5, 4, 4.5, 5, 5.5, 6])
        axs[metric_idx].set_xticklabels(['3', '3.5', '4', '4.5', '5', '5.5', '6'], fontsize=17)
        axs[metric_idx].set_yticks(k_folds)
        axs[metric_idx].set_yticklabels([f'K-fold = {k}' for k in k_folds], fontsize=17)
        axs[metric_idx].invert_yaxis()  
        #axs[metric_idx].set_ylabel(f'{metric}', fontsize=17)
        axs[metric_idx].set_title(f'{metric} Score Ranking', fontsize=20)
    
    axs[2].set_xlabel('Average Points (Log Scale)', fontsize=15)
    handles = [Line2D([0], [0], color=color, marker=marker, linestyle='', markersize=14) for color, marker in zip(colors, markers)]
    fig.legend(handles, models, fontsize=17, loc='upper center', ncol=8, bbox_to_anchor=(0.5, 1.02))

    plt.tight_layout()
    plt.show()

metrics = ['F1', 'ROC', 'MCC']

# Plot rankings for each method
for method in ['WF', 'BW', 'SW']:
    plot_ranking(combined_df, method, models, metrics, colors, markers)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D

wf_df = pd.concat([wf_f1_avg_points.assign(Metric='F1'), wf_roc_avg_points.assign(Metric='ROC'), wf_mcc_avg_points.assign(Metric='MCC')])
bw_df = pd.concat([bw_f1_avg_points.assign(Metric='F1'), bw_roc_avg_points.assign(Metric='ROC'), bw_mcc_avg_points.assign(Metric='MCC')])
sw_df = pd.concat([sw_f1_avg_points.assign(Metric='F1'), sw_roc_avg_points.assign(Metric='ROC'), sw_mcc_avg_points.assign(Metric='MCC')])

# Combine all DataFrames for easier plotting
combined_df = pd.concat([wf_df.assign(Method='WF'), bw_df.assign(Method='BW'), sw_df.assign(Method='SW')])

# Define models and their respective markers
models = combined_df['Model'].unique()
colors = ['#d7191c', '#fdae61', '#a6d96a', '#1a9641', '#2c7bb6', '#abd9e9', '#fdae61', '#f46d43']
markers = ['o', 's', 'd', '^', 'x', '*', '_', '+']

# Define k-folds
k_folds = [3, 4, 5, 6, 7, 8, 9]

def plot_ranking(df, method, models, metrics, colors, markers):
    fig, axs = plt.subplots(3, 1, figsize=(20, 30), sharex=True)
    
    for metric_idx, metric in enumerate(metrics):
        metric_df = df[(df['Method'] == method) & (df['Metric'] == metric)]
        for model in models:
            model_df = metric_df[metric_df['Model'] == model]
            axs[metric_idx].semilogx(model_df['points'], model_df['K-fold'], label=model, marker=markers[np.where(models == model)[0][0]], color=colors[np.where(models == model)[0][0]], linestyle='', markersize=20, alpha=0.7)
            # Add annotations with actual values
            for j, row in model_df.iterrows():
                axs[metric_idx].annotate(f'{row["points"]:.1f}', (row['points'], row['K-fold']), textcoords="offset points", xytext=(0, 12), ha='center', fontsize=14, color=colors[np.where(models == model)[0][0]])
        
        axs[metric_idx].set_xscale('log', base=2)
        axs[metric_idx].set_xlim(0.9, 7)  
        axs[metric_idx].set_xticks([1, 1.5, 2, 2.5, 3, 4, 5, 6, 8])
        axs[metric_idx].set_xticklabels(['1', '1.5', '2', '2.5', '3', '4', '5', '6', '7'], fontsize=17)
        axs[metric_idx].set_yticks(k_folds)
        axs[metric_idx].set_yticklabels([f'K-fold = {k}' for k in k_folds], fontsize=35)
        axs[metric_idx].invert_yaxis()  
        axs[metric_idx].set_title(f'{metric} Score Ranking', fontsize=35)
    
    axs[2].set_xlabel('Average Rank (Base 2)', fontsize=30)
    handles = [Line2D([0], [0], color=color, marker=marker, linestyle='', markersize=25) for color, marker in zip(colors, markers)]
    fig.legend(handles, models, fontsize=27, loc='upper center', ncol=8, bbox_to_anchor=(0.5, 1.03))

    plt.tight_layout()
    plt.savefig('/Users/6u0/Desktop/Article Submissions/Paper4/images/'f'{method}_rankings.png', dpi=300)
    plt.show()

metrics = ['F1', 'ROC', 'MCC']

# Plot rankings for each method
for method in ['WF', 'BW', 'SW']:
    plot_ranking(combined_df, method, models, metrics, colors, markers)

## Split Evaluation

In [ ]:
# trial for mcc

file_path = ''
df = pd.read_csv(file_path)

# Rank the MCC scores within each k-fold for each model
df['Rank'] = df.groupby(['K-fold', 'Model'])['mcc'].rank(method='min', ascending=False)

# Aggregate the ranks to compare split performance
rank_aggregation = df.groupby(['K-fold', 'split'])['Rank'].mean().reset_index()

rank_aggregation

In [ ]:
# mcc

file_path_bw = '/mcc/SortedMCC_tscv_bw.csv'
file_path_sw = '/mcc/SortedMCC_tscv_sw.csv'
file_path_wf = '/mcc/SortedMCC_tscv_wf.csv'

df_bw = pd.read_csv(file_path_bw)
df_sw = pd.read_csv(file_path_sw)
df_wf = pd.read_csv(file_path_wf)

def rank_and_aggregate(df):
    df['Rank'] = df.groupby(['K-fold', 'Model'])['mcc'].rank(method='min', ascending=False)
    rank_aggregation = df.groupby(['K-fold', 'split'])['Rank'].mean().reset_index()
    return rank_aggregation

rank_aggregation_wf = rank_and_aggregate(df_wf)
rank_aggregation_bw = rank_and_aggregate(df_bw)
rank_aggregation_sw = rank_and_aggregate(df_sw)

rank_aggregation_wf['Method'] = 'wf'
rank_aggregation_bw['Method'] = 'bw'
rank_aggregation_sw['Method'] = 'sw'

combined_ranks = pd.concat([rank_aggregation_wf, rank_aggregation_bw, rank_aggregation_sw], ignore_index=True)

combined_ranks
output_file_path = '/split_rank_mcc.csv'
combined_ranks.to_csv(output_file_path, index=False)

In [ ]:
# f1

file_path_bw = '/f1/SortedF1_tscv_bw.csv'
file_path_sw = '/f1/SortedF1_tscv_sw.csv'
file_path_wf = '/f1/SortedF1_tscv_wf.csv'

df_bw = pd.read_csv(file_path_bw)
df_sw = pd.read_csv(file_path_sw)
df_wf = pd.read_csv(file_path_wf)

def rank_and_aggregate(df):
    df['Rank'] = df.groupby(['K-fold', 'Model'])['f1_score'].rank(method='min', ascending=False)
    rank_aggregation = df.groupby(['K-fold', 'split'])['Rank'].mean().reset_index()
    return rank_aggregation

# Apply the function to all three dataframes
rank_aggregation_wf = rank_and_aggregate(df_wf)
rank_aggregation_bw = rank_and_aggregate(df_bw)
rank_aggregation_sw = rank_and_aggregate(df_sw)

rank_aggregation_wf['Method'] = 'wf'
rank_aggregation_bw['Method'] = 'bw'
rank_aggregation_sw['Method'] = 'sw'

combined_ranks = pd.concat([rank_aggregation_wf, rank_aggregation_bw, rank_aggregation_sw], ignore_index=True)

combined_ranks
output_file_path = '/Users/6u0/Desktop/Article Submissions/Paper4/final_results/split_rank_f1.csv'
combined_ranks.to_csv(output_file_path, index=False)

In [ ]:
# roc

file_path_bw = '/roc/SortedROC_tscv_bw.csv'
file_path_sw = '/roc/SortedROC_tscv_sw.csv'
file_path_wf = '/roc/SortedROC_tscv_wf.csv'

df_bw = pd.read_csv(file_path_bw)
df_sw = pd.read_csv(file_path_sw)
df_wf = pd.read_csv(file_path_wf)

def rank_and_aggregate(df):
    df['Rank'] = df.groupby(['K-fold', 'Model'])['roc_auc_score'].rank(method='min', ascending=False)
    rank_aggregation = df.groupby(['K-fold', 'split'])['Rank'].mean().reset_index()
    return rank_aggregation

rank_aggregation_wf = rank_and_aggregate(df_wf)
rank_aggregation_bw = rank_and_aggregate(df_bw)
rank_aggregation_sw = rank_and_aggregate(df_sw)

rank_aggregation_wf['Method'] = 'wf'
rank_aggregation_bw['Method'] = 'bw'
rank_aggregation_sw['Method'] = 'sw'

combined_ranks = pd.concat([rank_aggregation_wf, rank_aggregation_bw, rank_aggregation_sw], ignore_index=True)

combined_ranks
output_file_path = '/split_rank_roc.csv'
combined_ranks.to_csv(output_file_path, index=False)

In [ ]:
path = ''

mcc_file_path = path + 'split_rank_mcc.csv'
roc_file_path = path + 'split_rank_roc.csv'
f1_file_path = path + 'split_rank_f1.csv'

df_mcc = pd.read_csv(mcc_file_path)
df_roc = pd.read_csv(roc_file_path)
df_f1 = pd.read_csv(f1_file_path)

In [ ]:
sns.set(style="white")

plt.figure(figsize=(14, 8))
sns.boxplot(x='split', y='Rank', hue='Method', data=df_mcc)
plt.title('MCC: Distribution of Ranks for Each Split Across Different Methods and K-folds')
plt.xlabel('Split')
plt.ylabel('Rank')
plt.legend(title='Method')
plt.show()

plt.figure(figsize=(14, 8))
sns.lineplot(x='split', y='Rank', hue='Method', data=df_mcc, marker='o')
plt.title('MCC: Average Rank per Split for Each Method')
plt.xlabel('Split')
plt.ylabel('Average Rank')
plt.legend(title='Method')
plt.show()

plt.figure(figsize=(14, 8))
sns.barplot(x='split', y='Rank', hue='Method', data=df_mcc)
plt.title('MCC: Comparison of Average Ranks of Splits Across Methods and K-folds')
plt.xlabel('Split')
plt.ylabel('Average Rank')
plt.legend(title='Method')
plt.show()

heatmap_data = df_mcc.pivot_table(index='K-fold', columns=['Method', 'split'], values='Rank', aggfunc='mean')

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data, annot=True, cmap='viridis', linewidths=0.5)
plt.title('Heatmap of Split Rankings for MCC Across Methods and K-folds')
plt.xlabel('Method and Split')
plt.ylabel('K-fold')
plt.show()

In [ ]:
# heatmaps with standard colorbar:

import seaborn as sns
import matplotlib.pyplot as plt

def create_heatmap(df, title, vmin, vmax):
    heatmap_data = df.pivot_table(index='K-fold', columns=['Method', 'split'], values='Rank', aggfunc='mean')
    sns.heatmap(heatmap_data, fmt=".2f", annot=True, cmap='viridis', linewidths=0.5, annot_kws={"size": 17}, vmin=vmin, vmax=vmax)
    plt.title(title, fontsize=30)
    plt.xlabel('Method and Split', fontsize=20)
    plt.ylabel('K-fold', fontsize=20)

combined_df = pd.concat([df_mcc, df_roc, df_f1])
vmin = combined_df['Rank'].min()
vmax = combined_df['Rank'].max()

plt.figure(figsize=(20, 20))
gs = plt.GridSpec(3, 1, hspace=5)

plt.subplot(3, 1, 1)
create_heatmap(df_mcc, 'Split Rankings for MCC Across Methods and K-folds', vmin, vmax)

plt.subplot(3, 1, 2)
create_heatmap(df_roc, 'Split Rankings for ROC Across Methods and K-folds', vmin, vmax)

plt.subplot(3, 1, 3)
create_heatmap(df_f1, 'Split Rankings for F1 Across Methods and K-folds', vmin, vmax)

plt.tight_layout()
plt.savefig('/Users/6u0/Desktop/Article Submissions/Paper4/images/split_rankings.png', dpi=300)
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def create_heatmap(df, title, vmin, vmax, filename):
    plt.figure(figsize=(20, 6))  # Adjust the figsize as needed
    heatmap_data = df.pivot_table(index='K-fold', columns=['Method', 'split'], values='Rank', aggfunc='mean')
    sns.heatmap(heatmap_data, fmt=".2f", annot=True, cmap='viridis', linewidths=0.5, annot_kws={"size": 17}, vmin=vmin, vmax=vmax)
    #plt.title(title, fontsize=30)
    plt.xlabel('Method and Split', fontsize=20)
    plt.ylabel('K-fold', fontsize=20)
    plt.tight_layout()
    #plt.savefig(filename, dpi=300)
    plt.show()

combined_df = pd.concat([df_mcc, df_roc, df_f1])
vmin = combined_df['Rank'].min()
vmax = combined_df['Rank'].max()

create_heatmap(df_mcc, 'Split Rankings for MCC Across Methods and K-folds', vmin, vmax, '/Users/6u0/Desktop/Article Submissions/Paper4/images/split_rankings_mcc.png')
create_heatmap(df_roc, 'Split Rankings for ROC Across Methods and K-folds', vmin, vmax, '/Users/6u0/Desktop/Article Submissions/Paper4/images/split_rankings_roc.png')
create_heatmap(df_f1, 'Split Rankings for F1 Across Methods and K-folds', vmin, vmax, '/Users/6u0/Desktop/Article Submissions/Paper4/images/split_rankings_f1.png')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create separate heatmaps for each TSCV method, sharing a colorbar
def create_separate_heatmaps(df, metric_name, vmin, vmax):
    methods = ['bw', 'sw', 'wf']
    fig, axs = plt.subplots(1, 3, figsize=(20, 6), sharey=True)

    for i, method in enumerate(methods):
        method_data = df[df['Method'] == method].pivot(index='K-fold', columns='split', values='Rank')
        sns.heatmap(method_data, fmt=".2f", linewidths=0.5, ax=axs[i], vmin=vmin, vmax=vmax, annot=True, annot_kws={"size": 17},
                    cmap='viridis', cbar=i == 2)
        #axs[i].set_title(f'{method.upper()} - {metric_name}', fontsize=15)
        axs[i].set_xlabel('Split', fontsize=12)
        if i == 0:
            axs[i].set_ylabel('K-fold', fontsize=12)

    plt.tight_layout()
    plt.show()

#sns.heatmap(heatmap_data, fmt=".2f", annot=True, cmap='viridis', linewidths=0.5, annot_kws={"size": 17}, vmin=vmin, vmax=vmax)
    
vmin = combined_df['Rank'].min()  
vmax = combined_df['Rank'].max()  

create_separate_heatmaps(df_mcc, 'MCC', vmin, vmax)
create_separate_heatmaps(df_roc, 'ROC', vmin, vmax)
create_separate_heatmaps(df_f1, 'F1', vmin, vmax)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create separate heatmaps without titles
def create_separate_heatmaps_fixed_box_size(df, vmin, vmax, box_height=1, box_width=1):
    methods = ['bw', 'sw', 'wf']
    nrows = df['K-fold'].nunique()  # Get the number of unique K-folds (rows)
    ncols = df['split'].nunique()   # Get the number of splits (columns)

    fig, axs = plt.subplots(1, 3, figsize=(ncols * box_width * 3, nrows * box_height), sharey=True)

    for i, method in enumerate(methods):
        method_data = df[df['Method'] == method].pivot(index='K-fold', columns='split', values='Rank')

        sns.heatmap(method_data, ax=axs[i], vmin=vmin, vmax=vmax, annot=True, cmap='viridis',
                    cbar=i == 2, cbar_kws={"shrink": 0.8} if i == 2 else None, 
                    linewidths=0.5, square=True)

        axs[i].set_xlabel('Split', fontsize=12)

        if i == 0:
            axs[i].set_ylabel('K-fold', fontsize=12)

    plt.tight_layout()
    plt.show()

# Example of using the function for three metrics (MCC, ROC, F1)
vmin = combined_df['Rank'].min()  # Set vmin for consistent colorbar scaling
vmax = combined_df['Rank'].max()  # Set vmax for consistent colorbar scaling

# Generate heatmaps without titles
create_separate_heatmaps_fixed_box_size(df_mcc, vmin, vmax)
create_separate_heatmaps_fixed_box_size(df_roc, vmin, vmax)
create_separate_heatmaps_fixed_box_size(df_f1, vmin, vmax)
